In [1]:
import pandas as pd
import numpy as np
import os

from gensim.models import Word2Vec

from classes import *

%run functions.py

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler


In [3]:
# if data not saved as csv, run this
import os
if not os.path.exists('data/merged_titles_labels.csv'):
    df1 = pd.read_csv('../eda/small1/labeled.csv')
    df2 = pd.read_csv('../eda/small2/labeled.csv')
    df3 = pd.read_csv('../eda/small3/labeled.csv')
    df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
    df.to_csv('data/merged_titles_labels.csv', index=False)
    df.head()
else:
    df = pd.read_csv('data/merged_titles_labels.csv')
df.head()


,title,is_clickbait
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [4]:
%run functions.py
if not os.path.exists('data/preprocessed_titles_labels.pkl'):
    df = preprocess_title(df, verbose=True)
    df.to_pickle('data/preprocessed_titles_labels.pkl') 

else:
    df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,title,is_clickbait
0,"[house, dem, aide, , even, see, comey, letter...",1
1,"[flynn, hillary, clinton, big, woman, campus, ...",0
2,"[truth, might, get, fired]",1
3,"[fifteen, civilian, killed, single, usa, airst...",1
4,"[iranian, woman, jailed, fictional, unpublishe...",1


In [5]:
# read results from previous runs json file
result_path = 'results/word2vec_results_top25.json'
import json
word2vec_results = dict()
try:
    with open(result_path, 'r') as f:
        word2vec_results = json.load(f)
except:
    print("No previous results found")

# get all the models from the results
models_already_trained = []
for model in word2vec_results:
    models_already_trained.append(model)
models_already_trained = [model + ".model" for model in models_already_trained]
models_already_trained

No previous results found


[]

In [6]:
from tqdm.auto import tqdm
import copy as cp
import json

# import json results/not_trained_yet.json
with open('results/not_trained_yet.json', 'r') as f:
    not_trained_yet = json.load(f)

files = not_trained_yet['top_25']
files = [file for file in files if file.endswith('.model')]


# remove the files that have already been trained
files = [file for file in files if file not in models_already_trained]

files = sorted(files)
print(len(files))
files


25


['word2vec_vs1000_win7_sg0.model',
 'word2vec_vs1000_win7_sg1.model',
 'word2vec_vs1000_win8_sg0.model',
 'word2vec_vs1500_win3_sg1.model',
 'word2vec_vs1500_win4_sg0.model',
 'word2vec_vs1500_win4_sg1.model',
 'word2vec_vs1500_win5_sg0.model',
 'word2vec_vs1500_win5_sg1.model',
 'word2vec_vs1500_win6_sg0.model',
 'word2vec_vs1500_win6_sg1.model',
 'word2vec_vs1500_win7_sg0.model',
 'word2vec_vs1500_win7_sg1.model',
 'word2vec_vs1500_win8_sg0.model',
 'word2vec_vs1500_win8_sg1.model',
 'word2vec_vs2000_win3_sg1.model',
 'word2vec_vs2000_win4_sg0.model',
 'word2vec_vs2000_win5_sg0.model',
 'word2vec_vs2000_win5_sg1.model',
 'word2vec_vs2000_win6_sg0.model',
 'word2vec_vs2000_win6_sg1.model',
 'word2vec_vs2000_win7_sg0.model',
 'word2vec_vs2000_win7_sg1.model',
 'word2vec_vs2000_win8_sg1.model',
 'word2vec_vs2500_win3_sg0.model',
 'word2vec_vs2500_win3_sg1.model']

In [7]:
# tempw2v = Word2Vec.load('word2vec_models/word2vec_vs1000_win3_sg0.model')

In [8]:
# iterate over files in word2vec_models folder
for file in tqdm(files):
    # check if file is a .model file
    if file.endswith('.model'):

        # load model
        # path to model
        model_path = os.path.join('word2vec_models', file)

        properties = file.split('_')
        # print(properties)

        vector_size = int(properties[1][2:])
        window_size = int(properties[2][3:])
        is_skipgram = bool(int(properties[3][2:-6]))

        settings = {
            'model_path': model_path,
            'is_skipgram' : is_skipgram,
            'window_size' : window_size,
            'vector_size' : vector_size, 

        }
        # print(settings)

        model_w2v = Word2VecModel(settings)
        # get model name
        model_name = file.split('.')[0]

        title_vectors = [get_word_vectors(model_w2v, title, aggregation='mean') for title in df['title']]

        X = np.vstack(title_vectors)
        y = df['is_clickbait'].values
   

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42, shuffle=True)

        # scale data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # train models
        models = {
            'catboost': [
                CatBoostClassifier(verbose=False, random_state=1, ),
                CatBoostClassifier(verbose=False, random_state=2, ),
                CatBoostClassifier(verbose=False, random_state=3, ),
             
            ],

            'lightgbm': [
                LGBMClassifier(random_state=1, verbose=-1),
                LGBMClassifier(random_state=2, verbose=-1),
                LGBMClassifier(random_state=3 ,verbose=-1),
            
            ],
            
            'knn' : [
                KNeighborsClassifier(n_neighbors=3),
                KNeighborsClassifier(n_neighbors=4),
                KNeighborsClassifier(n_neighbors=5),
               
            ],

            'randomforest': [
                RandomForestClassifier(random_state=1,),
                RandomForestClassifier(random_state=2, ),
                RandomForestClassifier(random_state=3, ),
        

            ],

            'decisiontree': [
                DecisionTreeClassifier(random_state=1, ),
                DecisionTreeClassifier(random_state=2, ),
                DecisionTreeClassifier(random_state=3, ),
               
            ],
        }


        results = {}

        for model_type in tqdm(models, desc=f'Predictive Models for {model_name}'):
            f1_temp = list()
            auc_temp = list()
            # print(model_type)
            for submodel in tqdm(models[model_type], desc=f'{model_type} submodels'):
                submodel.fit(X_train_scaled, y_train)
                y_pred = submodel.predict(X_test_scaled)
                f1 = f1_score(y_test, y_pred)
                auc = roc_auc_score(y_test, y_pred)
                f1_temp.append(f1)
                auc_temp.append(auc)
                
            results[model_type] = {
                'f1': np.mean(f1_temp),
                'auc': np.mean(auc_temp),
                'f1_list': cp.deepcopy(f1_temp),
                'auc_list': cp.deepcopy(auc_temp),
            }
            
            
        word2vec_results[model_name] = results
        # save results to json
        with open(result_path, 'w') as f:
            json.dump(word2vec_results, f, indent=4)
        print(f'{model_name} done and updated json')





  0%|          | 0/25 [00:00<?, ?it/s]

Predictive Models for word2vec_vs1000_win7_sg0:   0%|          | 0/5 [00:00<?, ?it/s]

catboost submodels:   0%|          | 0/3 [00:00<?, ?it/s]

lightgbm submodels:   0%|          | 0/3 [00:00<?, ?it/s]

knn submodels:   0%|          | 0/3 [00:00<?, ?it/s]

...

catboost submodels:   0%|          | 0/3 [00:00<?, ?it/s]

lightgbm submodels:   0%|          | 0/3 [00:00<?, ?it/s]

knn submodels:   0%|          | 0/3 [00:00<?, ?it/s]

randomforest submodels:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import json

# read results from json file
with open('results/word2vec_results.json', 'r') as f:
    word2vec_results = json.load(f)

    
word2vec_results

{'word2vec_vs3000_win8_sg0': {'catboost': {'f1': 0.7943770191216275,
   'auc': 0.8285133923270581},
  'lightgbm': {'f1': 0.7816820783795684, 'auc': 0.8185836516262391},
  'knn': {'f1': 0.7273664318845559, 'auc': 0.7743116626186062},
  'randomforest': {'f1': 0.7565971581472596, 'auc': 0.8005246138556864},
  'decisiontree': {'f1': 0.648312463720043, 'auc': 0.7060215550558557}},
 'word2vec_vs2500_win4_sg0': {'catboost': {'f1': 0.7899571341090019,
   'auc': 0.8250251638898527},
  'lightgbm': {'f1': 0.779816513761468, 'auc': 0.8171465770595326},
  'knn': {'f1': 0.7328190743338009, 'auc': 0.7798611631524697},
  'randomforest': {'f1': 0.7563710499490317, 'auc': 0.8005087850690096},
  'decisiontree': {'f1': 0.6427280197206244, 'auc': 0.7001149966434287}},
 'word2vec_vs2500_win8_sg1': {'catboost': {'f1': 0.7920983318700614,
   'auc': 0.826754715255097},
  'lightgbm': {'f1': 0.7722719141323792, 'auc': 0.8115571765251608},
  'knn': {'f1': 0.7197853044604849, 'auc': 0.773053712996284},
  'randomfo

In [ ]:
# for each word2vec model calculate mean auc
word2vec_mean_auc = {}

for model in word2vec_results:
    # get results for each model
    results = word2vec_results[model]
    # get auc for each model
    aucs = [results[model]['auc'] for model in results]
    # calculate mean auc
    mean_auc = np.mean(aucs)
    # add to dict
    word2vec_mean_auc[model] = mean_auc

In [ ]:
# sort dict by mean auc in descending order
word2vec_mean_auc = dict(sorted(word2vec_mean_auc.items(), key=lambda item: item[1], reverse=True))
word2vec_mean_auc

In [ ]:
# for each word2vec model calculate mean auc
word2vec_mean_f1 = {}

for model in word2vec_results:
    # get results for each model
    results = word2vec_results[model]
    # get auc for each model
    f1s = [results[model]['f1'] for model in results]
    # calculate mean auc
    mean_f1 = np.mean(f1s)
    # add to dict
    word2vec_mean_f1[model] = mean_f1

In [ ]:
# sort dict by mean f1 in descending order
word2vec_mean_f1 = dict(sorted(word2vec_mean_f1.items(), key=lambda item: item[1], reverse=True))
word2vec_mean_f1

{'word2vec_vs100_win7_sg0': 0.7481139771677718,
 'word2vec_vs100_win5_sg0': 0.7464023223091918,
 'word2vec_vs100_win8_sg0': 0.7447431475432482,
 'word2vec_vs100_win6_sg0': 0.7437269708396137,
 'word2vec_vs2500_win8_sg0': 0.7424202842048666,
 'word2vec_vs100_win6_sg1': 0.7423892057677954,
 'word2vec_vs3000_win8_sg0': 0.7416670302506109,
 'word2vec_vs3000_win6_sg0': 0.7416515297374398,
 'word2vec_vs2000_win7_sg0': 0.7415567577421489,
 'word2vec_vs2500_win7_sg0': 0.7413774071614074,
 'word2vec_vs2500_win6_sg0': 0.7411358196375295,
 'word2vec_vs100_win3_sg0': 0.7409554745275384,
 'word2vec_vs3000_win5_sg0': 0.740845261061209,
 'word2vec_vs2000_win8_sg0': 0.7407646723882559,
 'word2vec_vs100_win8_sg1': 0.7407394944165616,
 'word2vec_vs250_win8_sg0': 0.7404521485178063,
 'word2vec_vs2500_win4_sg0': 0.7403383583747853,
 'word2vec_vs2000_win6_sg0': 0.7401628577314715,
 'word2vec_vs2500_win5_sg0': 0.7400008858023105,
 'word2vec_vs3000_win7_sg0': 0.7399683322448511,
 'word2vec_vs1500_win8_sg0': 

In [ ]:
# get first 10 models by auc and their mean auc
top_10_auc = dict(list(word2vec_mean_auc.items())[:10])

# get first 10 models by f1 and their mean f1
top_10_f1 = dict(list(word2vec_mean_f1.items())[:10])




# get intersection of top 10 models by auc and f1
top_10_auc_f1 = set(top_10_auc.keys()).intersection(set(top_10_f1.keys()))
top_10_auc_f1

{'word2vec_vs100_win5_sg0',
 'word2vec_vs100_win6_sg0',
 'word2vec_vs100_win6_sg1',
 'word2vec_vs100_win7_sg0',
 'word2vec_vs100_win8_sg0',
 'word2vec_vs2000_win7_sg0',
 'word2vec_vs2500_win8_sg0',
 'word2vec_vs3000_win6_sg0',
 'word2vec_vs3000_win8_sg0'}

In [ ]:
top_10_auc

{'word2vec_vs100_win7_sg0': 0.7911968235238132,
 'word2vec_vs100_win5_sg0': 0.7899047214230568,
 'word2vec_vs100_win8_sg0': 0.7887069489112742,
 'word2vec_vs100_win6_sg0': 0.7878347023739947,
 'word2vec_vs2500_win8_sg0': 0.786192899592545,
 'word2vec_vs100_win6_sg1': 0.786148329499342,
 'word2vec_vs3000_win6_sg0': 0.7856352883070729,
 'word2vec_vs3000_win8_sg0': 0.7855909750966892,
 'word2vec_vs100_win3_sg0': 0.7855303193339898,
 'word2vec_vs2000_win7_sg0': 0.7855182975876579}

In [ ]:
top_10_f1

{'word2vec_vs100_win7_sg0': 0.7481139771677718,
 'word2vec_vs100_win5_sg0': 0.7464023223091918,
 'word2vec_vs100_win8_sg0': 0.7447431475432482,
 'word2vec_vs100_win6_sg0': 0.7437269708396137,
 'word2vec_vs2500_win8_sg0': 0.7424202842048666,
 'word2vec_vs100_win6_sg1': 0.7423892057677954,
 'word2vec_vs3000_win8_sg0': 0.7416670302506109,
 'word2vec_vs3000_win6_sg0': 0.7416515297374398,
 'word2vec_vs2000_win7_sg0': 0.7415567577421489,
 'word2vec_vs2500_win7_sg0': 0.7413774071614074}

## WINNER - `word2vec_vs100_win7_sg0`
Since it was on average the best performing model for both auc and f1 on different predictive models, we will use this model to generate the word embeddings for the rest of the project.